In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from conf.conf_analysis import protca, dist
from plot.plot_utilities import edgeformat, xaxis, yaxis, savefig

In [ ]:
%store -r traj_ids_closed

In [ ]:
main = protca(traj_ids_closed, resids=[464, 549, 1250, 1348], close_after_init=False)

main.load_cainfo()
main.ca_coord_set

In [ ]:
site1_r1, site1_r2 = (464, 1348)
site2_r1, site2_r2 = (549, 1250)

# nbd1_r = [site1_r[0], site2_r[0]]
# nbd2_r = [site1_r[-1], site2_r[-1]]

# Dist1
xyz1_df = main.ca_coord_set.query("resid == @site1_r1")
xyz2_df = main.ca_coord_set.query("resid == @site1_r2")
dist_site1 = dist(xyz1_df[['x', 'y', 'z']].values, xyz2_df[['x', 'y', 'z']].values)

dist_df1 = pd.DataFrame()
dist_df1[['traj_id', 'timestep']] = xyz1_df[['traj_id', 'timestep']]
dist_df1[['r1', 'r2']] = [site1_r1, site1_r2]
dist_df1['distance'] = dist_site1


# Dist2
xyz1_df = main.ca_coord_set.query("resid == @site2_r1")
xyz2_df = main.ca_coord_set.query("resid == @site2_r2")
dist_site2 = dist(xyz1_df[['x', 'y', 'z']].values, xyz2_df[['x', 'y', 'z']].values)

dist_df2 = pd.DataFrame()
dist_df2[['traj_id', 'timestep']] = xyz2_df[['traj_id', 'timestep']]
dist_df2[['r1', 'r2']] = [site2_r1, site2_r2]
dist_df2['distance'] = dist_site2

fig, Axs = plt.subplots(5, 4, figsize=(12, 8), sharex=True, sharey=True, gridspec_kw={'hspace': 0.15, 'wspace': 0.1})

fulld = [0,8,11]
partiald = [1,3,4,6,9,10,12,13,15,16,17,19]

for i, t, axs in zip(np.arange(len(traj_ids)), traj_ids, Axs.flatten()):
    edgeformat(axs)
    dist1 = dist_df1.query('traj_id == @t')['distance'].values
    axs.plot(dist1)
    dist1_6msm = 10.92
    axs.plot([0,500], [dist1_6msm]*2, color='blue', ls='--')
    
    dist2 = dist_df2.query('traj_id == @t')['distance'].values
    axs.plot(dist2)
    dist2_6msm = 9.71
    axs.plot([0,500], [dist2_6msm]*2, color='red', ls='--')
    
    axs.set_xlim(0,500)
    xaxis(axs, title=None, min=0, max=500, step=250, labels=[0,0.5,1])
    axs.set_ylim(0,50)
    yaxis(axs, title=None, min=0, max=50, step=25)
    
    # Set background color
    if i in fulld:
        axs.set_facecolor('lightgreen')
    elif i in partiald:
        axs.set_facecolor('palegoldenrod')
    
background = fig.add_subplot(111)
# Hide everything except the axis labels
background.spines['top'].set_color('none')
background.spines['bottom'].set_color('none')
background.spines['left'].set_color('none')
background.spines['right'].set_color('none')
background.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
# Also set the background to completely transparent
background.patch.set_alpha(0.0)

background.set_ylabel("Distance [$\mathrm{\AA}$]", fontsize=16)
background.set_xlabel(r'Time [$\mathrm{\mu}$s]', fontsize=16)

# savefig("5uaksim_nbddist_vs_time.pdf")